In [1]:
import pandas as pd
import requests as req
import time
import geopy
from geopy.geocoders import Geolake
from geopy.extra.rate_limiter import RateLimiter

geopy.geocoders.options.default_timeout = 50
geolocator = Geolake(api_key='ZasbJHf0RtU0JpXS2IlG')

In [2]:
def get_areas(city_name, city_state, address, geolocator):
    
    #Format city name and state and Get entire page
    city_name = city_name.lower().replace(' ', '-')
    city_state = city_state.lower().replace(' ', '-')
    page_html = req.get("https://www.mapsofindia.com/pincode/india/{}/{}".format(city_state, city_name)).text
    
    #Get required table
    ts = page_html.find("<table><tr>")
    te = page_html.find("</table>", ts) + 8
    table_html = page_html[ts:te]
    
    #Convert to dataframe from html
    df = pd.read_html(table_html, header=[1], skiprows=0)[0]
    df = df.groupby('Pincode').count().reset_index().drop(columns=['Location', 'State', 'District'])
    df['address'] = df['Pincode'].map(str) + ", " + address #for geocoding-pandas integration w/o explicit loop
    
    #Add new lat and lng columns
    df['pc_lat'] = 0.0
    df['pc_lng'] = 0.0
    
#     geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)
#     df['loc'] = df['address'].apply(geocode)
    #Get lat-lng for each pincode
    for i, pin in zip(df.index, df['Pincode']):
        add = '{}, {}'.format(pin, address)
        print(i, add)
        pc_loc = geolocator.geocode({'zipcode': pin, 'city': city_name, 'country': 'India'})
        time.sleep(1)
        if(pc_loc != None):
            df.loc[i, 'pc_lat'] = pc_loc.latitude
            df.loc[i, 'pc_lng'] = pc_loc.longitude


    #Dislpay details for santity check
    print("Read Dataframe:")
    display(df.head())
    print("Rows: ", df.shape[0], "\nCols: ", df.shape[1])
    print("Data Types:", df.dtypes)
    
    return df

In [3]:
#Can be user input
#Hard-coded for simplicity
CITIES = [
    #{"name": "Kolkata", "state": "West Bengal"}
    {"name": "Mumbai", "state": "Maharashtra"}
]
COUNTRY = "India"

pin_df = None
#Get coordinates and pincodes of CITIES
for CITY in CITIES:
    
    #(1) Get Coordinates from Address using geolocator
    address = "{}, {}".format(CITY["name"], COUNTRY)
    CITY_loc = geolocator.geocode(address)
    CITY_lat = CITY_loc.latitude
    CITY_lng = CITY_loc.longitude
    print('The geograpical coordinate of {} are {}, {}.'.format(address, CITY_lat, CITY_lng))
    
    #(2) Get pincodes within cities by scraping www.mapsofindia.com
    pin_df = get_areas(CITY["name"], CITY["state"], address, geolocator)
    
    print("========================================================================")

The geograpical coordinate of Mumbai, India are 19.07283, 72.88261.
0 400001, Mumbai, India
1 400002, Mumbai, India
2 400003, Mumbai, India
3 400004, Mumbai, India
4 400005, Mumbai, India
5 400006, Mumbai, India
6 400007, Mumbai, India
7 400008, Mumbai, India
8 400009, Mumbai, India
9 400010, Mumbai, India
10 400011, Mumbai, India
11 400012, Mumbai, India
12 400013, Mumbai, India
13 400014, Mumbai, India
14 400015, Mumbai, India
15 400016, Mumbai, India
16 400017, Mumbai, India
17 400018, Mumbai, India
18 400019, Mumbai, India
19 400020, Mumbai, India
20 400021, Mumbai, India
21 400025, Mumbai, India
22 400026, Mumbai, India
23 400027, Mumbai, India
24 400028, Mumbai, India
25 400029, Mumbai, India
26 400030, Mumbai, India
27 400031, Mumbai, India
28 400032, Mumbai, India
29 400033, Mumbai, India
30 400034, Mumbai, India
31 400035, Mumbai, India
32 400037, Mumbai, India
33 400049, Mumbai, India
34 400050, Mumbai, India
35 400051, Mumbai, India
36 400052, Mumbai, India
37 400053, Mumbai

,Pincode,address,pc_lat,pc_lng
0,400001,"400001, Mumbai, India",18.6764,72.8819
1,400002,"400002, Mumbai, India",18.9750,72.8258
2,400003,"400003, Mumbai, India",18.9500,72.8333
3,400004,"400004, Mumbai, India",18.9500,72.8167
4,400005,"400005, Mumbai, India",18.9069,72.8106


Rows:  67 
Cols:  4
Data Types: Pincode      int64
address     object
pc_lat     float64
pc_lng     float64
dtype: object


In [4]:
pin_df.to_csv('mumbai_geolocation.csv')

# Rough 

In [ ]:
df = pd.DataFrame({'A': [10, 20, 30, 40, 40, 50],
                  'B': ['a', 'a', 'a', 'b', 'b', 'b'],
                  'C': ['a', 'b', 'b', 'a', 'c', 'a']})
display(df)
df1 = df.drop_duplicates(subset=['B', 'C'], keep='first')
display(df1)

df2 = df.drop_duplicates()
display(df2)